In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"manoachhossoudodo","key":"381c397cdbc7647bca372c074dffb7e4"}'}

In [3]:
import os

os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 600)


In [4]:
!pip install -q kaggle
!kaggle datasets download -d blastchar/telco-customer-churn
!unzip telco-customer-churn.zip

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
  0% 0.00/172k [00:00<?, ?B/s]
100% 172k/172k [00:00<00:00, 410MB/s]
Archive:  telco-customer-churn.zip
  inflating: WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [5]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/ChurnProject
!cp /content/WA_Fn-UseC_-Telco-Customer-Churn.csv /content/drive/MyDrive/ChurnProject/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/ChurnProject/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


L’**exploration des données (EDA)**, c’est l’étape où tu **comprends ton dataset** avant de construire un modèle.
C’est une étape **essentielle**, car elle te permet de :

✅ vérifier la qualité des données
✅ identifier les problèmes (valeurs manquantes, erreurs, etc.)
✅ comprendre la distribution des variables
✅ détecter les variables importantes
✅ préparer le dataset pour le modèle (encodage, nettoyage, etc.)

---

# 🧠 En clair, EDA = “je découvre ce que j’ai”

---

# 🧩 Ce que ça comprend généralement

## 1️⃣ Structure du dataset

* combien de lignes et colonnes ?
* types de données (numériques, catégorielles)

Exemple :

```python
df.info()
```

---

## 2️⃣ Statistiques de base (numériques)

* moyenne, écart-type, min/max, quartiles

```python
df.describe()
```

---

## 3️⃣ Valeurs manquantes

* quelles colonnes ont des valeurs manquantes ?

```python
df.isnull().sum()
```

---

## 4️⃣ Distribution de la cible

* combien de clients churn vs pas churn ?

```python
df['Churn'].value_counts()
```

---

## 5️⃣ Visualisations (optionnel mais utile)

* histogrammes
* barplots
* corrélations

Exemples :

```python
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df, x='Churn')
plt.show()
```

---

# 🎯 Pourquoi c’est crucial pour k-NN ?

KNN dépend des **distances**, donc :

* les variables doivent être **scalées**
* les variables catégorielles doivent être **encodées**
* les valeurs manquantes doivent être **traitées**
* les valeurs extrêmes (outliers) peuvent influencer les distances


In [7]:
df.info()
df.describe()
df.isnull().sum()
df['Churn'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


,count
Churn,
No,5174
Yes,1869


Super 👌
Les résultats montrent déjà des points très importants pour la suite.

---

# ✅ Ce que ça nous dit

## 1️⃣ Taille du dataset

* **7043 lignes**
* **21 colonnes**

---

## 2️⃣ Types de colonnes

* 2 colonnes numériques (`SeniorCitizen`, `tenure`)
* 1 colonne float (`MonthlyCharges`)
* **18 colonnes object (catégorielles)**

➡️ **On devra encoder beaucoup de variables** (OneHotEncoder).

---

## 3️⃣ Problème important : `TotalCharges` est en object

Même si `TotalCharges` est une valeur numérique, il est **au format texte**.

➡️ On doit convertir en numérique.

---

## 4️⃣ Cible (Churn) déséquilibrée

* **No : 5174**
* **Yes : 1869**

➡️ **Churn ≠ 50/50**
Donc il faudra faire attention aux métriques (accuracy seule peut être trompeuse).

---

# 🎯 Prochaine étape : nettoyer et préparer les données

## 1) Convertir `TotalCharges` en numérique

Il y a probablement des valeurs vides ou mal formatées.

On va faire :

```python
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.isnull().sum()
```

---

## 2) Traiter les valeurs manquantes (si il y en a)

Si on a des NaN dans `TotalCharges`, on peut :

* soit les supprimer
* soit les remplacer par la moyenne/median

Pour k-NN, mieux vaut supprimer si c’est peu.

---

## 3) Encodage des variables catégorielles

On va faire :

```python
X = pd.get_dummies(X, drop_first=True)
```

---

## 4) Scaling (obligatoire pour k-NN)

```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
```


In [8]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.isnull().sum()


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


# 🔥 Point clé

📌 **Il y a 11 valeurs manquantes dans `TotalCharges`**
Toutes les autres colonnes sont complètes.

---

# ✅ Que faire avec ces 11 lignes ?

Pour k-NN, c’est souvent **plus simple et propre de supprimer** ces lignes, car :

* c’est **très peu** (11 sur 7043 → 0.16%)
* k-NN ne supporte pas bien les valeurs manquantes
* on évite de biaiser avec une imputation

---

# 🧹 Étape suivante : supprimer les lignes avec NaN

Exécute :

```python
df = df.dropna()
df.isnull().sum()
```

---

📌 Ensuite, on fera :

* encodage des variables catégorielles
* scaling
* split train/test
* k-NN


In [9]:
df = df.dropna()
df.isnull().sum()


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


👉 **Pour faire la prédiction**, on se base sur **toutes les colonnes du tableau qui décrivent le client**, sauf celles qui n’apportent pas d’information utile (comme l’identifiant).

---

# ✅ On se base sur quoi exactement ?

## 🎯 **Variables d’entrée (features)**

Ce sont les colonnes qui décrivent le client et son comportement :

| Colonne          | Signification            | Exemple                          |
| ---------------- | ------------------------ | -------------------------------- |
| gender           | sexe                     | Male/Female                      |
| SeniorCitizen    | senior ?                 | 0 ou 1                           |
| Partner          | a un partenaire ?        | Yes/No                           |
| Dependents       | a des dépendants ?       | Yes/No                           |
| tenure           | durée d’abonnement       | 1 à 72 mois                      |
| PhoneService     | service téléphonique     | Yes/No                           |
| MultipleLines    | plusieurs lignes         | Yes/No/No phone service          |
| InternetService  | type d’internet          | DSL/Fiber/No                     |
| OnlineSecurity   | sécurité en ligne        | Yes/No                           |
| OnlineBackup     | sauvegarde en ligne      | Yes/No                           |
| DeviceProtection | protection du device     | Yes/No                           |
| TechSupport      | support technique        | Yes/No                           |
| StreamingTV      | streaming TV             | Yes/No                           |
| StreamingMovies  | streaming films          | Yes/No                           |
| Contract         | type de contrat          | Month-to-month/One year/Two year |
| PaperlessBilling | facturation électronique | Yes/No                           |
| PaymentMethod    | mode de paiement         | Electronic check, etc.           |
| MonthlyCharges   | montant mensuel          |                                  |
| TotalCharges     | montant total payé       |                                  |

➡️ **Ce sont toutes les variables que tu utilises pour prédire le churn.**

---

## ❌ Ce qu’on ne doit pas utiliser

### ❌ `customerID`

* C’est juste un identifiant unique
* Il n’a **aucune valeur prédictive**
* Il peut même “polluer” le modèle

### ❌ `Churn`

* C’est la **cible**, pas une feature
* On veut prédire cette colonne

---

# 🎯 Conclusion

👉 **On se base sur toutes les colonnes sauf :**

* `customerID`
* `Churn`

---

# 🔥 Ensuite, le modèle apprend une règle du type :

> Si le client a tel profil (tenure faible, contract month-to-month, payment method = electronic check, etc.), alors il est plus probable qu’il quitte.


In [10]:
#Séparer X et y
X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn'].map({'Yes': 1, 'No': 0})


# ✅ Variables catégorielles = variables qualitatives

Ce sont des colonnes qui **ne sont pas numériques** et qui représentent des **catégories**.

### Exemples dans ton dataset :

| Variable        | Type         | Exemple de valeurs                   |
| --------------- | ------------ | ------------------------------------ |
| gender          | catégorielle | Male, Female                         |
| Partner         | catégorielle | Yes, No                              |
| InternetService | catégorielle | DSL, Fiber optic, No                 |
| Contract        | catégorielle | Month-to-month, One year, Two year   |
| PaymentMethod   | catégorielle | Electronic check, Mailed check, etc. |

---

# 🔥 Différence avec variable numérique

### ✔ Variable numérique (quantitative)

* contient des nombres
* on peut faire des opérations mathématiques

Exemples :

* `tenure` (nombre de mois)
* `MonthlyCharges` (montant)
* `TotalCharges` (montant)

---

# ⚠️ Pourquoi ça pose un problème pour k-NN ?

k-NN calcule **des distances** (ex: distance euclidienne).
Donc il faut que toutes les variables soient numériques.

➡️ Une variable comme `gender` = "Male"/"Female" ne peut pas être utilisée directement.

---

# ✅ Donc on encode les variables catégorielles

### Exemple :

`gender` → on transforme en numérique :

* Male → 1
* Female → 0

Ou via **One-Hot Encoding** (plus robuste) :

```
gender_Male | gender_Female
     1      |      0
```

---

# 📌 Dans ton dataset, quelles sont les variables catégorielles ?

Tu as beaucoup de colonnes `object`, donc ce sont toutes des catégorielles :

* gender
* Partner
* Dependents
* PhoneService
* MultipleLines
* InternetService
* OnlineSecurity
* OnlineBackup
* DeviceProtection
* TechSupport
* StreamingTV
* StreamingMovies
* Contract
* PaperlessBilling
* PaymentMethod
* Churn (cible)

---

# ✔️ Et c’est pour ça qu’on fait :

```python
X = pd.get_dummies(X, drop_first=True)
```


In [11]:
X = pd.get_dummies(X, drop_first=True)

# ✅ Étape suivante : **Scaling (normalisation)**

⚠️ **Important pour k-NN** :
k-NN utilise la **distance**, donc les variables doivent être sur la même échelle.

Exemple :

* `MonthlyCharges` peut aller jusqu’à 100
* `tenure` peut aller jusqu’à 72

Si on ne scale pas, une variable peut dominer l’autre.

---

# 🔥 Scaling avec StandardScaler

```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
```

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# 🎯 À quoi sert `train_test_split` ?

Elle sert à **séparer les données en deux parties** :

## ✅ 1) **Train (entraînement)**

* `X_train`, `y_train`
* C’est sur ces données que le modèle **apprend**

## ✅ 2) **Test (évaluation)**

* `X_test`, `y_test`
* Le modèle **n’a jamais vu ces données**
* On les utilise pour **mesurer la performance réelle**

---

# 🧠 Pourquoi c’est indispensable ?

Si tu entraînes et testes sur les mêmes données, tu peux avoir un modèle qui **apprend par cœur** (overfitting) et qui ne fonctionne pas sur de nouvelles données.

⚠️ Donc le test doit être **indépendant**.

---

# 🔥 Ce que font les paramètres

```python
test_size=0.2
```

➡️ 20% des données vont dans le test
➡️ 80% dans l’entraînement

---

```python
random_state=42
```

➡️ permet de reproduire la même séparation à chaque exécution
➡️ sinon la séparation change à chaque fois

---

```python
stratify=y
```

➡️ garde la même proportion de `Churn` dans train et test
➡️ très important quand les classes sont déséquilibrées

Dans ton dataset :

* No : 5174
* Yes : 1869

Sans stratify, tu peux avoir un test avec très peu de “Yes”.

---

# 📌 Résumé

| Variable         | Rôle                            |
| ---------------- | ------------------------------- |
| X_train, y_train | données pour entraîner          |
| X_test, y_test   | données pour tester             |
| test_size=0.2    | 20% test                        |
| stratify=y       | proportion de classes conservée |

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
#Entraîner le k-NN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


KNeighborsClassifier()

In [15]:
#Prédiction sur le test
y_pred = knn.predict(X_test)


# ✅ `y_pred = knn.predict(X_test)`

### 🎯 **But : faire des prédictions**

* `knn` est ton modèle entraîné (après `knn.fit(X_train, y_train)`)
* `X_test` contient les **données que le modèle n’a jamais vues**
* `predict()` demande au modèle de **prédire la classe** pour chaque ligne de `X_test`

---

## 🔍 Ce que le modèle renvoie

Comme ton problème est une classification binaire (Churn ou pas), `predict()` renvoie :

* **0** → le client ne quitte pas
* **1** → le client quitte

Donc `y_pred` est un tableau de prédictions.

---

# 📌 Exemple simple

Si `X_test` contient 5 clients :

```python
y_pred = [0, 1, 0, 0, 1]
```

Cela veut dire :

* client 1 → pas churn
* client 2 → churn
* client 3 → pas churn
* client 4 → pas churn
* client 5 → churn

---

# 🧠 Pourquoi on fait ça ?

Pour comparer ensuite :

```python
y_test  (vraies valeurs)
y_pred  (valeurs prédites)
```

Et calculer des métriques comme :

* accuracy
* precision
* recall
* f1-score


In [16]:
#Évaluation du modèle
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm


array([[861, 172],
       [172, 202]])

# 🔥 Interprétation de ta matrice

```
[[861, 172],
 [172, 202]]
```

Elle se lit comme :

|                | **Pred No** | **Pred Yes** |
| -------------- | ----------- | ------------ |
| **Actual No**  | 861         | 172          |
| **Actual Yes** | 172         | 202          |

---

# ✅ Explication ligne par ligne

### 1️⃣ **True Negatives (TN) = 861**

* Clients qui **n’ont pas churné**
* Et le modèle a prédit **No**
* Donc c’est correct

### 2️⃣ **False Positives (FP) = 172**

* Clients qui **n’ont pas churné**
* Mais le modèle a prédit **Yes**
* Donc erreur (faux positif)

### 3️⃣ **False Negatives (FN) = 172**

* Clients qui **ont churné**
* Mais le modèle a prédit **No**
* Donc erreur importante (on n’a pas détecté le churn)

### 4️⃣ **True Positives (TP) = 202**

* Clients qui **ont churné**
* Et le modèle a bien prédit **Yes**

---

# 🎯 Ce que ça veut dire concrètement

Ton modèle fait **beaucoup d’erreurs** dans les deux sens :

* Il prédit trop souvent “Yes” (FP = 172)
* Il rate aussi beaucoup de churns (FN = 172)

---

# ⚠️ Pourquoi c’est important dans le churn ?

👉 En churn prediction, les **False Negatives** sont souvent les plus coûteux :

* tu rates un client qui va partir
* tu ne fais pas de stratégie de rétention

Donc on veut **minimiser FN** (maximiser recall sur la classe “Yes”).

---

# 🧠 Prochaine étape : metrics (precision/recall/f1)

# 🏦 Contexte réel : churn = client qui quitte

Imagine que tu es le responsable d’un service télécom.
Tu veux savoir **qui va quitter** pour essayer de les retenir.

---

# 📊 Ta matrice de confusion :

```
[[861, 172],
 [172, 202]]
```

---

## 🟢 1) True Negatives (TN) = 861

**Ils ne vont pas partir, et le modèle a bien dit “non”**

➡️ **Bonne nouvelle : 861 clients sont stables.**

📌 Exemple :

> Client A reste, et le modèle dit “il reste” → ok

---

## 🔴 2) False Positives (FP) = 172

**Le modèle dit “il va partir”, mais en vrai il reste**

➡️ **Le modèle s’inquiète pour 172 clients inutilement.**

📌 Exemple :

> Client B reste, mais le modèle dit “il va partir”
> Tu vas peut-être lui envoyer une offre, alors qu’il n’en a pas besoin.

🧾 Conséquence :
Tu dépenses des ressources pour rien.

---

## ⚠️ 3) False Negatives (FN) = 172

**Le modèle dit “il ne va pas partir”, mais en vrai il part**

➡️ **C’est le pire cas en churn prediction.**

📌 Exemple :

> Client C part, mais le modèle dit “il reste”
> Tu ne le contactes pas, tu ne fais pas d’offre, et tu le perds.

💸 Conséquence :
Tu perds de l’argent parce que tu n’as pas anticipé le départ.

---

## 🟢 4) True Positives (TP) = 202

**Le modèle dit “il va partir”, et en vrai il part**

➡️ **C’est exactement ce qu’on veut détecter.**

📌 Exemple :

> Client D part, et le modèle a bien détecté
> Tu peux le contacter pour le retenir.

---

# 🎯 En résumé (terre à terre)

| Situation | Ce que le modèle a dit | Ce qui s’est passé | Résultat                         |
| --------- | ---------------------- | ------------------ | -------------------------------- |
| TN        | Il reste               | Il reste           | OK                               |
| FP        | Il part                | Il reste           | Mauvais, mais pas catastrophique |
| FN        | Il reste               | Il part            | Très mauvais (on perd un client) |
| TP        | Il part                | Il part            | Très bon (on peut agir)          |

---

# 🚨 Le vrai problème ici

Tu as **172 clients qui partent mais que tu n’as pas détectés** (FN).

➡️ Ce sont des clients **que tu perds sans rien faire.**

---

# 🔥 Prochaine étape

On va maintenant calculer des métriques comme :

* **Recall (rappel)** : capacité à détecter les clients qui partent
* **Precision** : quand le modèle dit “il part”, est-ce vrai ?

Et ensuite on va **optimiser k** pour améliorer tout ça.


In [17]:
#Prochaine étape : metrics (precision/recall/f1)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.83      0.83      1033
           1       0.54      0.54      0.54       374

    accuracy                           0.76      1407
   macro avg       0.69      0.69      0.69      1407
weighted avg       0.76      0.76      0.76      1407



# ✅ `classification_report` fait quoi ?

Il calcule plusieurs métriques importantes pour **chaque classe** :

### 🔹 Pour la classe “0” (No churn)

### 🔹 Pour la classe “1” (Yes churn)

Et il affiche :

* **Precision**
* **Recall**
* **F1-score**
* **Support** (nombre d’exemples)

---

# 📌 Ce que signifient ces métriques (en langage simple)

---

## 🔵 Precision (Précision)

👉 Sur tous les clients que le modèle a dit “oui il va partir”, combien étaient vraiment partis ?

📌 Exemple :

* Le modèle dit “oui” pour 100 clients
* 60 étaient vraiment partis
  ➡️ Precision = 60%

**Donc precision = “fiabilité quand il dit oui”**

---

## 🔴 Recall (Rappel)

👉 Sur tous les clients qui sont réellement partis, combien le modèle a réussi à détecter ?

📌 Exemple :

* 100 clients sont réellement partis
* le modèle en détecte 70
  ➡️ Recall = 70%

**Donc recall = “capacité à ne pas rater les churns”**

---

## 🟣 F1-score

C’est la moyenne harmonique entre precision et recall.

📌 Utile quand les classes sont déséquilibrées (comme ici).

---

# 📌 Support

C’est juste le nombre d’exemples réels dans chaque classe.

---

# 🎯 Pourquoi c’est important pour le churn ?

👉 On veut surtout **maximiser le recall sur la classe “Yes”**, parce que rater un churn = perte de client.

---

# 🟢 Exemple concret du rapport

Quand tu exécutes :

```python
print(classification_report(y_test, y_pred))
```

Tu obtiens quelque chose comme :

```
              precision    recall  f1-score   support
0 (No)          0.83        0.83     0.83      1033
1 (Yes)         0.54        0.54     0.54       374
```

➡️ Ici tu vois que **pour les churns** (1) c’est faible → on doit améliorer.


# 📌 Interprétation du rapport

## ✅ Classe 0 (No churn)

* **Precision = 0.83**
  Quand le modèle dit “non”, il a raison 83% du temps.
* **Recall = 0.83**
  Il détecte correctement 83% des clients qui restent.
* **F1 = 0.83**
  C’est cohérent et bon.

➡️ Donc le modèle est **très bon pour prédire les clients qui restent**.

---

## ⚠️ Classe 1 (Yes churn)

* **Precision = 0.54**
  Quand le modèle dit “oui, il va partir”, il a raison seulement 54% du temps.
* **Recall = 0.54**
  Il détecte seulement 54% des clients qui partent.
* **F1 = 0.54**
  C’est moyen / faible.

➡️ Le modèle **rate trop de churns** et se trompe souvent quand il dit churn.

---

# 🎯 Pourquoi c’est important ?

Dans le churn, on veut surtout :

✅ **un bon recall sur la classe 1**
car rater un churn = perte d’argent.

---

# 🔥 Ce que ça veut dire en termes concrets

Sur 374 clients qui ont réellement quitté :

* le modèle en détecte **54%** → environ **202 clients**
* donc **172 churns sont ratés** (FN)

👉 C’est exactement ce que tu as vu dans la matrice de confusion.

---

# 📌 Score global

* **Accuracy = 0.76**
  Le modèle est correct 76% du temps

Mais attention :
➡️ L’accuracy n’est pas suffisante quand les classes sont déséquilibrées (comme ici).

---

# ✅ Prochaine étape : optimiser le modèle

On va maintenant :

### 1) **optimiser le paramètre k**

pour trouver le meilleur k possible

### 2) **essayer d’améliorer le recall**

sur la classe 1 (churn)

---

# 🚀 Optimisation de k (prochaine étape)


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors': range(1, 31)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)

print("Best k:", grid.best_params_)
print("Best f1:", grid.best_score_)


Best k: {'n_neighbors': 25}
Best f1: 0.5848004301529848


Ton meilleur **k** trouvé est **25** et le meilleur **F1-score** est **0.5848**.

---

# ✅ Ce que ça veut dire

## ✔️ Best k = 25

Le modèle fonctionne **mieux** quand il regarde **25 voisins** pour décider.

📌 Avec k plus petit, le modèle est trop sensible (bruit).
📌 Avec k plus grand, il devient plus stable.

---

## ✔️ Best f1 = 0.5848

Le F1 est passé de **0.54 à 0.58**, donc c’est une amélioration.

➡️ On a gagné en capacité à détecter le churn tout en gardant une bonne précision.

---

# 🎯 Maintenant on réentraîne avec k=25

---

## ✅ Code :

```python
knn_best = KNeighborsClassifier(n_neighbors=25)
knn_best.fit(X_train, y_train)

y_pred_best = knn_best.predict(X_test)
```

---

# 📌 Ensuite on vérifie les résultats :

```python
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))
```


In [19]:
#Maintenant on réentraîne avec k=25
knn_best = KNeighborsClassifier(n_neighbors=25)
knn_best.fit(X_train, y_train)

y_pred_best = knn_best.predict(X_test)


In [20]:
#Ensuite on vérifie les résultats :
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))


[[884 149]
 [160 214]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      1033
           1       0.59      0.57      0.58       374

    accuracy                           0.78      1407
   macro avg       0.72      0.71      0.72      1407
weighted avg       0.78      0.78      0.78      1407

